<a href="https://colab.research.google.com/github/abarna1822/mobile_point/blob/main/index_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.0 MB/s eta 0:00:00


In [2]:
!ngrok config add-authtoken 2ydudH1tDLF866IfVZAhRPOMJwM_3Din1BECgtMsNDgfMbFdj


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [3]:
!pip install reportlab

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 15.3 MB/s eta 0:00:00


In [4]:
code = '''
import streamlit as st
from datetime import datetime
import pandas as pd
from reportlab.pdfgen import canvas
import os

st.set_page_config(page_title="Mobile Point", layout="wide")

st.markdown("<h1 style='text-align: center; color: #00a8ff;'>📱 MOBILE POINT</h1>", unsafe_allow_html=True)

# === GLOBAL STATE ===
if 'data' not in st.session_state:
    st.session_state.data = []
    st.session_state.token_counter = 1

# === Utility Functions ===
def format_duration(seconds):
    mins, secs = divmod(seconds, 60)
    hours, mins = divmod(mins, 60)
    return f"{int(hours):02d}:{int(mins):02d}:{int(secs):02d}"

def update_dashboard():
    total_tokens = st.session_state.token_counter - 1
    mobiles_inside = sum(1 for d in st.session_state.data if d['out_time'] is None)
    durations = [(d['out_time'] - d['in_time']).total_seconds() for d in st.session_state.data if d['out_time']]
    avg_duration = format_duration(sum(durations) / len(durations)) if durations else "00:00:00"
    return total_tokens, mobiles_inside, avg_duration

def refresh_df():
    df = []
    for d in st.session_state.data:
        df.append({
            "Token": d['token'],
            "Roll Number": d['roll_number'],
            "In-Time": d['in_time'].strftime("%Y-%m-%d %H:%M:%S"),
            "Out-Time": d['out_time'].strftime("%Y-%m-%d %H:%M:%S") if d['out_time'] else "N/A",
            "Duration": format_duration((d['out_time'] - d['in_time']).total_seconds()) if d['out_time'] else "-"
        })
    return pd.DataFrame(df)

def generate_pdf(token, roll_number, in_time, out_time):
    filename = f"receipt_token_{token}.pdf"
    c = canvas.Canvas(filename)
    c.setFont("Helvetica", 14)
    c.drawString(100, 750, "📄 Mobile Point - Receipt")
    c.line(100, 740, 500, 740)
    c.drawString(100, 710, f"Token Number: {token}")
    c.drawString(100, 690, f"Roll Number: {roll_number}")
    c.drawString(100, 670, f"In Time: {in_time.strftime('%Y-%m-%d %H:%M:%S')}")
    c.drawString(100, 650, f"Out Time: {out_time.strftime('%Y-%m-%d %H:%M:%S')}")
    duration = out_time - in_time
    c.drawString(100, 630, f"Duration: {format_duration(duration.total_seconds())}")
    c.save()
    return filename

# === Input Area ===
if 'roll' not in st.session_state:
    st.session_state.roll = ""

roll = st.text_input("Enter Roll Number", value=st.session_state.roll, key="roll_input")

col1, col2, col3 = st.columns([1, 1, 1])
pdf_path = None

with col1:
    if st.button("Allocate Slot"):
        if not roll.strip():
            st.warning("Please enter roll number.")
        else:
            for d in st.session_state.data:
                if d['roll_number'] == roll and d['out_time'] is None:
                    st.info(f"Mobile already allocated to {roll}.")
                    break
            else:
                st.session_state.data.append({
                    'token': st.session_state.token_counter,
                    'roll_number': roll,
                    'in_time': datetime.now(),
                    'out_time': None
                })
                st.session_state.token_counter += 1
                st.success(f"Slot allocated for {roll}.")

with col2:
    if st.button("Deallocate Slot"):
        for d in st.session_state.data:
            if d['roll_number'] == roll and d['out_time'] is None:
                d['out_time'] = datetime.now()
                st.success(f"Mobile returned for roll {roll}. Generating receipt...")
                pdf_path = generate_pdf(d['token'], d['roll_number'], d['in_time'], d['out_time'])
                break
        else:
            st.info(f"No active allocation found for roll {roll}.")

with col3:
    if st.button("Clear Entry"):
        st.session_state.roll = ""  # Reset the input field

# === Dashboard ===
st.markdown("---")
st.markdown("### 📊 Dashboard")
total, inside, avg = update_dashboard()
col1, col2, col3 = st.columns(3)
col1.metric("Total Tokens", total)
col2.metric("Mobiles Inside", inside)
col3.metric("Average Duration", avg)

# === PDF Link ===
if pdf_path and os.path.exists(pdf_path):
    with open(pdf_path, "rb") as f:
        st.download_button("📥 Download PDF Receipt", f, file_name=pdf_path, mime="application/pdf")

# === Data Table ===
st.markdown("---")
st.markdown("### 📋 History")
df = refresh_df()
st.dataframe(df, use_container_width=True)
'''

with open("index.py", "w") as f:
    f.write(code)


In [5]:
# Kill previous Streamlit instances
!pkill streamlit

# Start Streamlit in background
!streamlit run index.py &>/content/log.txt &

# Connect ngrok (corrected)
from pyngrok import ngrok
public_url = ngrok.connect(addr="8501", proto="http")
print(f"✅ App is live at: {public_url}")

✅ App is live at: NgrokTunnel: "https://69ae-34-41-142-168.ngrok-free.app" -> "http://localhost:8501"
